# `xtensor` basics features

A brief overview of xtensor basic usage. Most of the examples are provided by the [official documentation](https://xtensor.readthedocs.io/en/latest/).

Including the necessary libs:

In [1]:
#include <iostream>
#include <typeinfo>

#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"  // this one is necessary for a proper display in notebooks
#include "xtensor/xview.hpp"
#include "xtensor/xmath.hpp"

## Basic `xarray` manipulations

Declaring arrays:

In [2]:
xt::xarray<double> arr1 {
    {1, 2, 3},
    {2, 5, 7},
    {2, 5, 7}
};

In [3]:
arr1

1.,2.,3.
2.,5.,7.
2.,5.,7.


In [4]:
xt::xarray<double> arr2{
    {5, 6, 7}
};

Below we perform a sum of the second line from `arr1` with `arr2`:

In [5]:
xt::xarray<double> result = xt::view(arr1, 1) + arr2;

std::cout << result << std::endl;

{{  7.,  11.,  14.}}


Be aware about the use of views. C++ `rvalues` are temporary, so the combination with `xexpression`s could be dangerous:

In [6]:
auto result_2 = xt::view(arr1, 1) + arr2;

std::cout << result_2 << std::endl;

{{ 7.,  8.,  9.}}


This occurs because `result_2` is not a `xarray<T>` object! It's inferred as `xexpression` (which does not hold any value):

In [7]:
std::cout << typeid(result_2).name();

N2xt9xfunctionINS_6detail4plusEJNS_5xviewIRNS_16xarray_containerINS_7uvectorIdSaIdEEELNS_11layout_typeE1ENS_7svectorImLm4ESaImELb1EEENS_22xtensor_expression_tagEEEJiEEERKSD_EEE

So it doesn't work as one must expected because it avoids evaluation at intermediate results. A proper way to fix it could be:

In [8]:
xt::xarray<double> arr1_1 = xt::view(arr1, 1);
auto result_3 = arr1_1 + arr2;

std::cout << result_3 << std::endl;

{{  7.,  11.,  14.}}


Another interesting thing we can do is reshaping. Let's declare another array. Now, an array with integer entries (note the Template input):

In [9]:
xt::xarray<int> arr3 {
    {1, 2, 3, 4, 5, 6, 7, 8, 9}
};

`xt::array<T>` has a convenience method for reshaping, which is used as follows:

In [10]:
arr3.reshape({3, 3});

std::cout << arr3;

{{1, 2, 3},
 {4, 5, 6},
 {7, 8, 9}}

It is very similar to `numpy.ndarray` counterpart. Note that the new shape must match the size of original array entries.

Access index is pretty straightforward. Let's consider `arr1`:

In [11]:
std::cout << arr1(0, 1);

2

In [12]:
arr1

1.,2.,3.
2.,5.,7.
2.,5.,7.


Or, with an 1D array:

In [13]:
arr3

1,2,3
4,5,6
7,8,9


In [14]:
xt::xarray<int> arr4 {
    {1, 2, 3, 4}
};

std::cout << arr4(2);

3

Now, let's use an universal function (`xt::pow`) to demonstrate the broadcasting feature. Firstly, let's declare two arrays with different template inputs and sizes:

In [15]:
xt::xarray<double> arr5 {
    {1.0, 2.0, 3.0}
};

xt::xarray<unsigned int> arr6 {
    {4, 5, 6, 7}
};

Now, we transform `arr6` in a column array:

In [16]:
arr6.reshape({4, 1});

In [17]:
arr6

4
5
6
7


Note that reshaping is performed inline! What happens if we use `xt::pow` with these two arrays? Let's see:

In [18]:
xt::xarray<double> result_4 = xt::pow(arr5, arr6);

std::cout << result_4;

{{    1.,    16.,    81.},
 {    1.,    32.,   243.},
 {    1.,    64.,   729.},
 {    1.,   128.,  2187.}}

Nice! However, be aware that the dimensions must be compatible. For example, if `arr6.reshape({1, 4})` was performed, the broadcasting wouldn't proceed.

## Expressions and lazy evaluation

In order to understand the conceptions, let's declare 3 new arrays:

In [19]:
xt::xarray<double> x {
    {2, 2, 4}
};

xt::xarray<double> y {
    {1, 0, 1}
};

xt::xarray<double> z {
    {0.3, 0.5, 0.7}
};

### Expressions

An expression can be build by operating the arrays inferred by `auto` such as

In [20]:
auto f = x + y * xt::sin(z);

You can use it to compose in a more complex expression:

In [21]:
auto f2 = y + 2 * xt::cos(f);

The expression avoids evaluation at intermediate results. It improves the performance as near to if you had written a simple loop. If you want to evaluate, just assign it to a `lvalue` variable (better saying, to a container):

In [22]:
xt::xarray<double> result_eval = y + 2 * xt::cos(f);

std::cout << result_eval;

{{-0.325857, -0.832294,  0.863763}}

### Lazy evaluation

This feature allows to compute the expression results only for the indices of interest. Let's consider now larger arrays:

In [23]:
#include "xtensor/xrandom.hpp" // to use pseudo-random generator

// Note that this is an expression
auto random_basis = xt::random::rand<double>({100000, 1});

xt::xarray<double> a = random_basis;
xt::xarray<double> b = random_basis;

Note that `a` and `b` are large, with 100000 entries. If we want only a certain input, we can evaluate only at the right index. Consider that we have the following expression:

In [24]:
auto f_large = xt::cos(a) + xt::sin(b);

If we only want two values, say, at 1200 and 2500, we only need to evaluate:

In [25]:
double first_res = f_large(1200);

and

In [26]:
double second_res = f_large(2500);

Only the above two values have been computed!

### Forcing evaluation

If, for any reason, you need to evaluate an `xexpression`, you can usa `xt::eval`. It can return an rvalue to a newly allocated container or a reference to a container. In order to avoid copies, you should use `auto&&`, the universal reference on LHS. For example:

In [27]:
xt::xarray<double> array_a = {1, 2, 3};
xt::xarray<double> array_b = {3, 2, 1};

and we define the following xexpression (which is not evaluated):

In [28]:
auto calculation = array_a + array_b;

Now we can get an `xarray` rvalue container as:

In [29]:
auto&& container_1 = xt::eval(calculation);

std::cout << container_1;

{ 4.,  4.,  4.}

### Expression interface

All `xexpression`s in `xtensor` provide the following convenience interfaces:

#### Shape

* `dimension()`: returns the number of dimensions;

* `shape()`: returns the shape of the expression.

In [30]:
#include <vector>

using array_type = xt::xarray<double>;
using shape_type = array_type::shape_type;

shape_type shape = {3, 2, 4};
array_type arr7(shape);

Let's check the dimension:

In [31]:
std::cout << arr7.dimension();

3

Now the shape (a little more tricky):

In [32]:
const shape_type& s = arr7.shape();

auto res_check = s == shape;

In [33]:
res_check

true

Be aware that the method `shape()` do not return readable shape format as in `numpy`. See below:

In [34]:
arr7.shape()

@0x7fc444000790

which is a memory reference (not sure, I need to confirm it).

#### Accessing